<a href="https://colab.research.google.com/github/firdasabaniyah/Submission-Akhir-Pengembangan-Machine-Learning/blob/main/cobaUntitled8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NAMA : FIRDA SA'BANIYAH**

**ID : M495Y1035**

**KELAS : M02**

In [1]:
! pip install -q kaggle

In [2]:
! chmod 600 /content/kaggle.json

In [3]:
! KAGGLE_CONFIG_DIR=/content kaggle datasets download -d alessiocorrado99/animals10

 98% 576M/586M [00:03<00:00, 189MB/s]
100% 586M/586M [00:03<00:00, 198MB/s]


In [4]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [5]:
import os
import zipfile
import shutil

In [6]:
local_zip = '/content/animals10.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

In [7]:
dir_utama = os.path.join('/tmp/raw-img')
print(os.listdir(dir_utama))

['pecora', 'ragno', 'cavallo', 'gallina', 'cane', 'scoiattolo', 'farfalla', 'gatto', 'mucca', 'elefante']


In [8]:
ignore_dir = ['gallina', 'cane', 'scoiattolo', 'farfalla', 'gatto', 'mucca', 'elefante']

for dir in ignore_dir:
  path = os.path.join(dir_utama, dir)
  shutil.rmtree(path)

print(os.listdir(dir_utama))

['pecora', 'ragno', 'cavallo']


In [10]:
from PIL import Image
samples = 0

for x in os.listdir(dir_utama):
  dir = os.path.join('/tmp/raw-img', x)
  y = len(os.listdir(dir))
  print(x+':', y)
  samples = samples + y
  
  image_name = os.listdir(dir)
  for z in range(3):
    image_path = os.path.join(dir, image_name[z])
    image = Image.open(image_path)
    print('-',image.size)
  print('\n')


print('\nTotal Keseluruhan sampel:', samples)

pecora: 1820
- (640, 426)
- (640, 426)
- (300, 203)


ragno: 4821
- (300, 200)
- (640, 451)
- (300, 200)


cavallo: 2623
- (300, 233)
- (300, 214)
- (300, 225)



Total Keseluruhan sampel: 9264


In [11]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    validation_split = 0.2)

In [13]:
train_generator = train_datagen.flow_from_directory(
    dir_utama,
    target_size=(150, 150),
    batch_size=128,
    class_mode='categorical',
    subset='training')
 
validation_generator = train_datagen.flow_from_directory(
    dir_utama,
    target_size=(150, 150),
    batch_size=128,
    class_mode='categorical',
    subset='validation')

Found 7412 images belonging to 3 classes.
Found 1852 images belonging to 3 classes.


In [14]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [15]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [16]:
class Callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.92 and logs.get('val_accuracy') > 0.85):
      print("Training dihentikan karena akurasi pada training set sudah mencapai 0.92 dan validation set sudah mencapai 0.85")
      self.model.stop_training = True

callbacks = Callback()